### Logistic Regression - Label Classification

In [41]:
import sys
sys.path.append('../../../../')

# Import libraries
from utils.score import fold_cross_validate
from utils.print import print_cross_validation_scores
from constant.columns import FEATURES, LABEL, SIZE, TARGET_Y, MODE, IS_SINGLE_INPUT

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [42]:
# Import data
feat = FEATURES
if IS_SINGLE_INPUT ==  False:
    WINDOW_SIZE = f"{SIZE}_{TARGET_Y}"
    if MODE == 1:
        WINDOW_SIZE = f"{SIZE}_{TARGET_Y}_normalize"
    if MODE == 2:
        WINDOW_SIZE = f"{SIZE}_{TARGET_Y}_normalize_std"

    print(f'WINDOW_SIZE: {WINDOW_SIZE}')
    # Get features with window size
    expanded_features = []
    for feature in FEATURES:
        for i in range(SIZE):
            expanded_features.append(f'{feature}_{i}')
    
    feat = expanded_features

# Read dataset
df = pd.read_csv(f'./source/dataset_smote.csv')
X, y = df[feat], df[LABEL]

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

WINDOW_SIZE: 10_10


#### Cross Validation

In [43]:
# Define the model
clf_cv = LogisticRegression()
scores = fold_cross_validate(clf_cv, X_test, y_test)

# Print scores
print_cross_validation_scores(scores)

MACRO:
Precision: 0.8380 (0.0184)
Recall: 0.8380 (0.0172)
F1 score: 0.8368 (0.0173)
---------------------------------
MICRO:
Precision: 0.8374 (0.0171)
Recall: 0.8374 (0.0171)
F1 score: 0.8374 (0.0171)
---------------------------------


In [44]:
# # Define the optimized model (previous research)
# clf_cv = LogisticRegression(penalty="l1", C=1, solver="liblinear", max_iter=100, class_weight=None, fit_intercept=False, multi_class="auto")
# scores = fold_cross_validate(clf_cv, X_test, y_test)

# # Print scores
# print_cross_validation_scores(scores)